In [2]:
import torch
from torch import nn

In [3]:
net = nn.Sequential(nn.Linear(4,8), nn.ReLU(), nn.Linear(8,1))
X = torch.rand(size=(2,4))
net(X)

tensor([[0.2877],
        [0.3135]], grad_fn=<AddmmBackward0>)

In [4]:
# 获取第二个全连接层的参数
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.2081, -0.1404,  0.1633,  0.1303, -0.0941,  0.1256,  0.1339,  0.3209]])), ('bias', tensor([0.0913]))])


In [5]:
# 每一个参数都表示为参数类的一个实例
# 参数类是复合的对象，包含值，梯度和额外信息
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.0913], requires_grad=True)
tensor([0.0913])


In [6]:
net[2].weight.grad == None

True

**打印神经网络 net 的第一层的所有参数的名称和形状。**   

named_parameters() 是一个方法，它返回一个迭代器，包含模块的所有参数的名称和值。  
[(name, param.shape) for name, param in net[0].named_parameters()] 是一个列表推导式，它创建了一个列表，列表中的每个元素都是一个元组，元组的第一个元素是参数的名称，第二个元素是参数的形状。   
*是一个解包运算符，它将列表中的每个元素作为一个单独的参数传递给 print() 函数   

In [7]:
# 打印net第一层的所有参数的名称和形状
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


**state_dict()**  
state_dict() 是 PyTorch 中 nn.Module 类的一个方法，它返回一个字典，包含了模型的所有参数。  
   
在这个字典中，键是参数的名称，值是参数的值（一个张量）。这个字典包含了模型的所有权重和偏置，以及优化器的状态（如果你在优化器上调用了 state_dict()）。  
  
state_dict() 是 PyTorch 中保存和加载模型的关键。你可以使用 torch.save() 函数保存 state_dict() 返回的字典，然后使用 torch.load() 函数和 load_state_dict() 方法来加载模型

In [8]:
net.state_dict()['2.bias'].data

tensor([0.0913])

In [9]:
def block1():
    return nn.Sequential(nn.Linear(4,8), nn.ReLU(),
                         nn.Linear(8,4), nn.ReLU())
    
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}',block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4,1))
rgnet(X)

tensor([[0.3119],
        [0.3119]], grad_fn=<AddmmBackward0>)

In [10]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [11]:
rgnet[0][3][2].weight.data

tensor([[-0.0691,  0.2002,  0.0645,  0.2545,  0.0671, -0.0146,  0.1196,  0.0428],
        [ 0.0592, -0.1218,  0.1680, -0.1286, -0.3440, -0.0478, -0.0686, -0.2244],
        [ 0.0280, -0.2244, -0.0664,  0.0880, -0.1802, -0.1288,  0.2058,  0.3246],
        [ 0.1022,  0.3491, -0.2154,  0.2694, -0.3002, -0.2038, -0.2686,  0.3202]])

In [12]:
def init_normal(m):
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight, mean=0,std=0.01)
        nn.init.zeros_(m.bias)
# apply函数会递归地遍历所有模块并对参数执行init_normal
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0095,  0.0154, -0.0170, -0.0035]), tensor(0.))

In [13]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight,2)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data,net[0].bias.data

(tensor([[2., 2., 2., 2.],
         [2., 2., 2., 2.],
         [2., 2., 2., 2.],
         [2., 2., 2., 2.],
         [2., 2., 2., 2.],
         [2., 2., 2., 2.],
         [2., 2., 2., 2.],
         [2., 2., 2., 2.]]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0.]))

In [14]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
# 对每个层定义初始化        
net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)
        

tensor([ 0.4765,  0.3398, -0.6886,  0.5697])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [15]:
# 自定义初始化
def my_init(m):
    if type(m)==nn.Linear:
        print("Init",*[(name, param.shape) for name, param in m.named_parameters()][0])
        # 使用均匀分布来初始化模型的权重
        nn.init.uniform_(m.weight,-10,10)
        # 将权重的绝对值小于 5 的部分（对应的布尔值为 False）乘以 0
        # m.weight.data.abs()>=5 是一个布尔张量，它的每个元素表示对应的权重的绝对值是否大于或等于5
        m.weight.data *= m.weight.data.abs()>=5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000,  0.0000, -9.0929,  0.0000],
        [ 6.4736, -0.0000, -5.6634, -5.9763]], grad_fn=<SliceBackward0>)

In [16]:
net[0].weight.data[:] += 1
net[0].weight.data[0,1] = 42
net[0].weight.data[0]

tensor([ 1.0000, 42.0000, -8.0929,  1.0000])

In [19]:
shared = nn.Linear(8,8)
net = nn.Sequential(nn.Linear(4,8), nn.ReLU(),
                   shared, nn.ReLU(),
                   shared, nn.ReLU(),
                   nn.Linear(8,1)
                   )

net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0,0] = 100
print(net[2].weight.data[0]==net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
